<a href="https://colab.research.google.com/github/tintenderete/TFM-Algoritmo-con-redes-evolutivas/blob/main/test_evolutivo_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PLANTEAMIENTO INICIAL

Planteamiento inicial, algunas cosas cambiaran durante el desarrollo.

Esta es una primera iteracion de un modelo evolutivo, se intentara simplificar lo maximo posible para ganar velocidad.

## PUNTO 1 : MODELOS

Los modelos invertiran durante 1 año, cada 30 timesteps.

EL modelo evolutivo se comparara con un modelo no evolutivo.  El no evolutivo , en este test, no es realmente un modelo pero lo ignoraremos.

El modelo no evolutivo sera una frontera de marko que invertira en los X activos con mejor ratio de sharpe. 

El modelo evolutivo hara lo similar con la excepcion de que seleccionara los X mejores, en funcion de su output.

## PUNTO 1 : DATOS 

Usaremos los precios de cierre diarios del IBEX 35 descargados con la api yahoo finance. 

Si algun activo da problemas sera eliminado, y los nans seran rellenas con fills. Se instentara usar un periodo historico que de pocos problemas.

Usaremos 3 años para train y 1 año para test.
Los datos se generaran en periodos pseudo-mensuales, es decir, empezando en el primer dia cada 30 dias.

Cada ejemplo de entrenamiento constara de 60 periodos diarios  , tendra 3 canales (para cada activo) : 
            
            - retornos logaritmicos diarios

            - media movil (de retornos logaritmicos) de los ultimos 60 dias

            - desviacion tipica movil (de retornos logaritmicos) de los ultimos 60 dias


x.shape => (BATCH, 60, 35,  3)

## PUNTO 2 : FUNCION LISTAS

Para rankear los activos usaremos el ratio de sharpe.

y.shape => (BATCH, 35, )

## PUNTO 3 : INICIALIZACION DE INDIVIDUOS

Se utilizara el metodo desarrollado en el notebook A_individuos_3

Se usaran pocos individuos, la intencion es utilizar alrededor de 100.

##PUNTO 4 : EVALUACION / FITNESS

In [ ]:
"""

Se entrenara en batches de 1 dato

Se evaluara despues de entrenar con todos los datos.

La funcion de coste utilizada sera : 

def perdida_listas(ranking_real, ranking_predicho):
    diferencias = tf.abs(ranking_real - ranking_predicho)
    diferencias_ponderadas = diferencias / (ranking_real + 1e-7)  
    diferencias_ponderadas = diferencias / (ranking_predicho + 1e-7) 
    return perdida_total

esta funcion castiga mas cuando los peores activos son clasificados como mejores.

La razon inicial para empezar por esta funcion es que , dado que nuestro objetivo es calcular una frontera de marko en los X mejores, 
realmente no necesitamos que esten fielmente ordenados, lo que queremos es asegurarnos de que los activos interesantes sean clasificados entre los X mejores.
     

"""

##PUNTO 5 : SELECCION 

In [ ]:
"""

Seleccionaremos los X mejores en funcion de la funcion de coste.

¿ Emparejamiento ? 

Teniendo un pull de X individuos, se elegiran al azar, sin reemplazo y de 2 en 2,  de dentro del pull, 

"""

## PUNTO 6 : CRUCE

In [ ]:
"""

Utilizaremos un cruce uniforme para favorecer la biodiversidad. 

"""

## PUNTO 7 : MUTACION

In [ ]:
"""

Por el momento no se introducira mutacion

"""

## PUNTO 8 : REEMPLAZO

In [ ]:
"""

Pasaran a la siguiente generacion:

  Los X mejores .

  Los hijos de los X mejores. ( 2 hijos por pareja)

  X nuevos individuos random ( Inmigracion )


"""